<a href="https://colab.research.google.com/github/zacharymoskow/reel2recipe.com/blob/main/Steve_Retail_Area_Search_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas requests

In [ ]:
pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display

# API endpoints for the datasets
dob_api_filtered = "https://data.cityofnewyork.us/resource/ipu4-2q9a.json"
pluto_api_filtered = "https://data.cityofnewyork.us/resource/64uk-42ks.json"

# Function to load the dataset with pagination
def load_data(api_url, params, required_columns=None):
    all_data = []
    offset = 0
    while True:
        params['$offset'] = offset
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        data = response.json()
        if not data:
            break
        all_data.extend(data)
        offset += len(data)
        if len(data) < params['$limit']:
            break
    df = pd.DataFrame(all_data)
    if required_columns:
        df = df[required_columns]
    return df

# Load the DOB dataset for the Bronx
dob_columns = [
    'filing_date', 'house__', 'street_name', 'block', 'lot',
    'owner_s_business_name', 'owner_s_first_name',
    'owner_s_last_name', 'owner_s_phone__'
]
dob_params = {
    'borough': 'BRONX',
    'permit_type': 'NB',
    '$limit': 50000
}
dob_data = load_data(dob_api_filtered, dob_params, dob_columns)

# Convert filing_date to datetime
dob_data['filing_date'] = pd.to_datetime(dob_data['filing_date'], errors='coerce')

# Sort DOB data by filing_date in descending order
dob_data = dob_data.sort_values(by='filing_date', ascending=False)

# Load the PLUTO dataset for the Bronx
pluto_columns = ['block', 'lot', 'retailarea', 'bldgarea', 'garagearea', 'officearea']
pluto_params = {
    'borough': 'BX',
    '$limit': 50000
}
pluto_data = load_data(pluto_api_filtered, pluto_params, pluto_columns)

# Convert block and lot to consistent formats for merging
def format_block_lot(df):
    df['block'] = df['block'].astype(str).str.lstrip('0')  # Strip leading zeros
    df['lot'] = df['lot'].astype(str).str.lstrip('0')      # Strip leading zeros
    return df

dob_data = format_block_lot(dob_data)
pluto_data = format_block_lot(pluto_data)

# Print some information about the datasets
print(f"DOB data shape: {dob_data.shape}")
print(f"PLUTO data shape: {pluto_data.shape}")
print("\nDOB data 'block' and 'lot' sample:")
print(dob_data[['block', 'lot']].head())
print("\nPLUTO data 'block' and 'lot' sample:")
print(pluto_data[['block', 'lot']].head())

# Define the dropdown for selecting weeks or months
time_unit_dropdown = widgets.Dropdown(
    options=['Weeks', 'Months'],
    description='Time Unit:',
    disabled=False
)

# Define the input for the number of weeks or months
time_value_input = widgets.IntText(
    value=2,
    description='Value:',
    disabled=False
)

# Display the dropdowns
display(time_unit_dropdown, time_value_input)

def filter_data(time_unit, time_value):
    if time_unit == 'Weeks':
        time_delta = timedelta(weeks=time_value)
    elif time_unit == 'Months':
        time_delta = timedelta(days=30 * time_value)
    else:
        print("Invalid time unit selected.")
        return

    date_filter = datetime.now() - time_delta
    print(f"Filtering DOB data from {date_filter} to {datetime.now()}")

    # Filter DOB data by filing date
    filtered_dob = dob_data[dob_data['filing_date'] > date_filter]
    print(f"Filtered DOB data has {filtered_dob.shape[0]} rows")

    # Debug: Print unique block and lot values from filtered DOB data
    print("Unique block and lot values from filtered DOB data:")
    print(filtered_dob[['block', 'lot']].drop_duplicates().head())

    # Debug: Print unique block and lot values from PLUTO data
    print("Unique block and lot values from PLUTO data:")
    print(pluto_data[['block', 'lot']].drop_duplicates().head())

    # Merge with PLUTO data on block and lot
    merged_data = pd.merge(
        filtered_dob,
        pluto_data[['block', 'lot', 'retailarea', 'bldgarea', 'garagearea', 'officearea']],
        on=['block', 'lot'],
        how='left'
    )
    print(f"Merged data has {merged_data.shape[0]} rows")

    # Check for null values in retailarea, garagearea, and officearea
    null_retail = merged_data['retailarea'].isnull().sum()
    null_garage = merged_data['garagearea'].isnull().sum()
    null_office = merged_data['officearea'].isnull().sum()
    print(f"Number of properties without retail area: {null_retail}")
    print(f"Number of properties without garage area: {null_garage}")
    print(f"Number of properties without office area: {null_office}")

    # Convert retailarea, garagearea, and officearea to numeric, replacing any non-numeric values with NaN
    merged_data['retailarea'] = pd.to_numeric(merged_data['retailarea'], errors='coerce')
    merged_data['garagearea'] = pd.to_numeric(merged_data['garagearea'], errors='coerce')
    merged_data['officearea'] = pd.to_numeric(merged_data['officearea'], errors='coerce')

    # Filter to include properties with retail area > 0 OR garage area > 5000 OR office area > 2500
    properties_with_criteria = merged_data[
        (merged_data['retailarea'] > 0) |
        (merged_data['garagearea'] > 5000) |
        (merged_data['officearea'] > 2500)
    ]
    print(f"Filtered properties with retail area > 0, garage area > 5000, or office area > 2500: {properties_with_criteria.shape[0]} rows")

    if not properties_with_criteria.empty:
        avg_retail_area = properties_with_criteria['retailarea'].mean()
        max_retail_area = properties_with_criteria['retailarea'].max()
        avg_garage_area = properties_with_criteria['garagearea'].mean()
        max_garage_area = properties_with_criteria['garagearea'].max()
        avg_office_area = properties_with_criteria['officearea'].mean()
        max_office_area = properties_with_criteria['officearea'].max()
        print(f"Average retail area: {avg_retail_area:.2f} sq ft")
        print(f"Maximum retail area: {max_retail_area:.2f} sq ft")
        print(f"Average garage area: {avg_garage_area:.2f} sq ft")
        print(f"Maximum garage area: {max_garage_area:.2f} sq ft")
        print(f"Average office area: {avg_office_area:.2f} sq ft")
        print(f"Maximum office area: {max_office_area:.2f} sq ft")
    else:
        print("No properties matching the criteria found.")

    # Select the required columns
    output_columns = [
        'filing_date', 'house__', 'street_name', 'block', 'lot', 'bldgarea',
        'retailarea','garagearea', 'officearea', 'owner_s_business_name',
        'owner_s_first_name', 'owner_s_last_name', 'owner_s_phone__'
    ]

    # Sort by filing date (most recent first)
    properties_with_criteria = properties_with_criteria.sort_values('filing_date', ascending=False)

    # Add row numbers starting from 1
    properties_with_criteria = properties_with_criteria.reset_index(drop=True)
    properties_with_criteria.index += 1

    # Display the total number of properties
    total_properties = properties_with_criteria.shape[0]
    print(f"Number of new build properties in the Bronx with filing dates in the last {time_value} {time_unit.lower()} and matching the criteria: {total_properties}")

    # Display the result
    display(properties_with_criteria[output_columns])

# Button to trigger the filtering
button = widgets.Button(description="Filter Data")

def on_button_clicked(b):
    filter_data(time_unit_dropdown.value, time_value_input.value)

button.on_click(on_button_clicked)
display(button)


DOB data shape: (36996, 9)
PLUTO data shape: (89603, 6)

DOB data 'block' and 'lot' sample:
      block lot
36993  4168  13
36990  2850  63
36991  3600   4
36992  3934  37
36989  3600  40

PLUTO data 'block' and 'lot' sample:
  block lot
0  3813  42
1  3812  44
2  3812  60
3  3814  55
4  3812  64


Dropdown(description='Time Unit:', options=('Weeks', 'Months'), value='Weeks')

IntText(value=2, description='Value:')

Button(description='Filter Data', style=ButtonStyle())

Filtering DOB data from 2024-06-05 00:00:33.494592 to 2024-08-04 00:00:33.494611
Filtered DOB data has 56 rows
Unique block and lot values from filtered DOB data:
      block lot
36993  4168  13
36990  2850  63
36991  3600   4
36992  3934  37
36989  3600  40
Unique block and lot values from PLUTO data:
  block lot
0  3813  42
1  3812  44
2  3812  60
3  3814  55
4  3812  64
Merged data has 56 rows
Number of properties without retail area: 18
Number of properties without garage area: 18
Number of properties without office area: 18
Filtered properties with retail area > 0, garage area > 5000, or office area > 2500: 8 rows
Average retail area: 1029.88 sq ft
Maximum retail area: 3317.00 sq ft
Average garage area: 5110.50 sq ft
Maximum garage area: 22662.00 sq ft
Average office area: 6066.25 sq ft
Maximum office area: 16472.00 sq ft
Number of new build properties in the Bronx with filing dates in the last 2 months and matching the criteria: 8


,filing_date,house__,street_name,block,lot,bldgarea,retailarea,garagearea,officearea,owner_s_business_name,owner_s_first_name,owner_s_last_name,owner_s_phone__
1,2024-07-03,414,EAST 152 STREET,2374,40,40237,0.0,0.0,10480.0,NEL 1120 LLC,SHAYA,SEIDENFELD,7188588400
2,2024-07-02,3073,BARKER AVE,4543,27,20225,0.0,0.0,2743.0,2257 GRABD AVE KKC,ELENA,BOROKHOVICH,3478660561
3,2024-07-02,1223,SPOFFORD AVE,2738,2,234313,0.0,22662.0,12890.0,NYCEDC,JHAELEN,HERNANDEZ-ELI,2123123800
4,2024-06-28,3545,WHITE PLAINS ROAD,4643,48,20907,3317.0,0.0,0.0,3545 WHITE PLAINS ROAD DEVELOPME,HAIM,LEVY,7184656000
5,2024-06-26,2495,SEDGWICK AVE,3237,9,120781,0.0,18222.0,16472.0,"SKF DEVELOPMENT, LLC",KIM,TASHER,2126041062
6,2024-06-22,2680,MORRIS AVENUE,3316,11,42649,1605.0,0.0,2845.0,2680 MORRIS LLC,SAKANDER,RAJA,9146648444
7,2024-06-14,3545,WHITE PLAINS ROAD,4643,48,20907,3317.0,0.0,0.0,3545 WHITE PLAINS ROAD DEVELOPME,HAIM,LEVY,7184656000
8,2024-06-12,1374,BLONDELL AVE,4139,106,34521,0.0,0.0,3100.0,1400 BLONDELL LLC,DAN,SHAVOLIAN,2126616300
